# **Urban Computing - (mini) Lab 2**

This lab provides tips for using Pandas dataframe more efficiently.

# How best to fill a Pandas dataframe?

When working with Pandas dataframes, you of course sometimes have to add some rows with data to it, either to fill an empty dataframe or add to an existing one. But are there worse and better ways to do it? Using the techniques introduced in Lab 1, we can investigate this quickly.

Get some data:

In [ ]:
# !pip install foursquare

import pandas as pd
import foursquare as fsq
pd.options.display.width = 1000
pd.options.display.max_columns = 0

# Please copy paste your client_id and client_secret from your web foursquare-app
cl_id='your client ID'
cl_sec='your client secret'

# Construct the client object
client = fsq.Foursquare(client_id=cl_id, client_secret=cl_sec)

# Use your Client and request data from foursquare
data = client.venues.search(params={'ll': '52.159536, 4.491366', 'query': 'bar', 
                                    'intent': 'browse', 'radius':3000, 'limit':50})



## First attempt:

In [ ]:
# Build a function
def venue_scraper_v1(data):
    df = pd.DataFrame(columns = ['name', 'latitude', 'longitude', 'distance', 'address'])
    
    for i in data['venues']:
        df.append({
            'name': i['name'],
            'latitude': i['location']['lat'],
            'longitude': i['location']['lng'],
            'distance': i['location']['distance'],
            'address': i['location']['formattedAddress'][0]
        }, ignore_index=True)
    
    return df

In [ ]:
print(venue_scraper_v1(data).head())

Empty DataFrame
Columns: [name, latitude, longitude, distance, address]
Index: []


Why didn't it work? Because `df.append()` returns a new dataframe!

## Second attempt

In [ ]:
# Build a function
def venue_scraper_v2(data):
    df = pd.DataFrame(columns = ['name', 'latitude', 'longitude', 'distance', 'address'])
    
    for i in data['venues']:
        df = df.append({
            'name': i['name'],
            'latitude': i['location']['lat'],
            'longitude': i['location']['lng'],
            'distance': i['location']['distance'],
            'address': i['location']['formattedAddress'][0]
        }, ignore_index=True)
    
    return df

In [ ]:
print(venue_scraper_v2(data).head())

                   name   latitude  longitude distance            address
0    Cosmo Blow Dry Bar  52.159146   4.490836       56  Aalmarkt 22 (V&D)
1  Bar-Dancing De Kroon  52.163177   4.494783      467     Langegracht 65
2          M Noodle Bar  52.162870   4.484963      573     Beestenmarkt 4
3           Rooftop Bar  52.164953   4.488021      644          Nederland
4    Bar Bistro Raphael  52.159177   4.491167       42      Nieuwe Rijn 1


Okay, it works, but is it the best way? We're generating a new dataframe for each row we're appending. That doesn't sound efficient.

## Third attempt

In [ ]:
# Build a function
def venue_scraper_v3(data):

    # Initialize a dictionary of lists
    d = {'name' : [], 'lat':[], 'lng': [], 'dist': [], 'address': [], 'genre': []}
    
    # Fill the dictionary from the API results
    for i in range(len(data['venues'])):
        d['name'].append(data['venues'][i]['name'])
        d['lat'].append(data['venues'][i]['location']['lat'])
        d['lng'].append(data['venues'][i]['location']['lng'])
        d['dist'].append(data['venues'][i]['location']['distance'])
        if 'address' in data['venues'][i]['location'].keys():
            d['address'].append(data['venues'][i]['location']['address'])
        else:
            d['address'].append('NA')
        if data['venues'][i]['categories'] != []:
            d['genre'].append(data['venues'][i]['categories'][0]['pluralName'])
        else:
            d['genre'].append('NA')
            
    # Construct a dataframe from the dictionary-of-lists
    df = pd.DataFrame.from_dict(d)
    return df

In [ ]:
print(venue_scraper_v3(data).head())

                   name        lat  ...         address                 genre
0    Cosmo Blow Dry Bar  52.159146  ...     Aalmarkt 22  Salons / Barbershops
1  Bar-Dancing De Kroon  52.163177  ...  Langegracht 65              Gay Bars
2          M Noodle Bar  52.162870  ...  Beestenmarkt 4         Noodle Houses
3           Rooftop Bar  52.164953  ...              NA                    NA
4    Bar Bistro Raphael  52.159177  ...   Nieuwe Rijn 1                  Bars

[5 rows x 6 columns]


## Fourth variation

In [ ]:
# Build a function
def venue_scraper_v4(data):
    df = pd.DataFrame(columns = ['name', 'latitude', 'longitude', 'distance', 'address'])
    
    batch = []
    for i in data['venues']:
        batch.append(pd.Series([
            i['name'], 
            i['location']['lat'],
            i['location']['lng'],
            i['location']['distance'],
            i['location']['formattedAddress'][0]
        ], index=df.columns))
    df = df.append(batch, ignore_index=True)
    
    return df

In [ ]:
print(venue_scraper_v4(data).head())

                   name   latitude  longitude distance            address
0    Cosmo Blow Dry Bar  52.159146   4.490836       56  Aalmarkt 22 (V&D)
1  Bar-Dancing De Kroon  52.163177   4.494783      467     Langegracht 65
2          M Noodle Bar  52.162870   4.484963      573     Beestenmarkt 4
3           Rooftop Bar  52.164953   4.488021      644          Nederland
4    Bar Bistro Raphael  52.159177   4.491167       42      Nieuwe Rijn 1


## Comparison

In [ ]:
%timeit venue_scraper_v2(data)

1 loop, best of 3: 208 ms per loop


In [ ]:
%timeit venue_scraper_v3(data)

1000 loops, best of 3: 1.03 ms per loop


In [ ]:
%timeit venue_scraper_v4(data)

100 loops, best of 3: 11.8 ms per loop


Remember: 1 ms = 1000 µs

## What's the underlying idea?
Pandas uses Numpy arrays under the hood, which are very fast because every cell in the array is right next to the previous one. So reading from one cell to the next is a very simple and fast operation.

Pandas is in column-major order, which means that the data is stored in column-arrays in memory. So to add a row, you'd have to add a cell to the end of each column. And check if there's actually room there in memory to make the columns longer, because the cells of the column have to remain contiguous. 

Doing that again and again for each row is not efficient. So the *ideal* approach is to create a dataframe from all your data as a set of columns. If that's not an option, for example because you're handling some kind of incoming data stream, then you can still achieve a speedup by adding multiple rows as a batch.

(Of course, in practice, very often you're getting your data from a CSV because it doesn't make sense to download it again and again from an API if you're fine-tuning code that's not actually related to connecting to the API. And Pandas has excellent CSV import abilities to efficiently load big files.)